In [1]:
from __future__ import print_function, unicode_literals, absolute_import, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from tifffile import imread
from csbdeep.utils import download_and_extract_zip_file, axes_dict, plot_some, plot_history
from csbdeep.utils.tf import limit_gpu_memory
from csbdeep.io import load_training_data
from csbdeep.models import Config, CARE


ModuleNotFoundError: No module named 'tifffile'

In [2]:
raw_data = RawData.from_folder (
    basepath    = 'data/nucleo',
    source_dirs = ['in'],
    target_dir  = 'gt',
    axes        = 'CYX',
)

NameError: name 'RawData' is not defined

# Paul generated 200 patches from the same image 

In [3]:
X, Y, XY_axes = create_patches (
    raw_data            = raw_data,
    patch_size          = (128,128),
    n_patches_per_image = 1,
    save_file           = 'data/my_training_data.npz',
#     transforms= None
    
)

NameError: name 'create_patches' is not defined

In [ ]:
for i in range(2):
    plt.figure(figsize=(16,4))
    sl = slice(8*i, 8*(i+1)), 0
    plot_some(X[sl],Y[sl],title_list=[np.arange(sl[0].start,sl[0].stop)])
    plt.show()
None;

In [8]:
(X,Y), (X_val,Y_val), axes = load_training_data('data/my_training_data.npz', validation_split=0.1, verbose=True)

c = axes_dict(axes)['C']
n_channel_in, n_channel_out = X.shape[c], Y.shape[c]

number of training images:	 37
number of validation images:	 4
image size (2D):		 (256, 256)
axes:				 SYXC
channels in / out:		 1 / 1


In [9]:
config = Config(axes, n_channel_in, n_channel_out, probabilistic=True, train_steps_per_epoch=400)
print(config)
vars(config)

Config(axes='YXC', n_channel_in=1, n_channel_out=1, n_dim=2, probabilistic=True, train_batch_size=16, train_checkpoint='weights_best.h5', train_checkpoint_epoch='weights_now.h5', train_checkpoint_last='weights_last.h5', train_epochs=100, train_learning_rate=0.0004, train_loss='laplace', train_reduce_lr={'factor': 0.5, 'patience': 10, 'min_delta': 0}, train_steps_per_epoch=400, train_tensorboard=True, unet_input_shape=(None, None, 1), unet_kern_size=5, unet_last_activation='linear', unet_n_depth=2, unet_n_first=32, unet_residual=True)


{'n_dim': 2,
 'axes': 'YXC',
 'n_channel_in': 1,
 'n_channel_out': 1,
 'train_checkpoint': 'weights_best.h5',
 'train_checkpoint_last': 'weights_last.h5',
 'train_checkpoint_epoch': 'weights_now.h5',
 'probabilistic': True,
 'unet_residual': True,
 'unet_n_depth': 2,
 'unet_kern_size': 5,
 'unet_n_first': 32,
 'unet_last_activation': 'linear',
 'unet_input_shape': (None, None, 1),
 'train_loss': 'laplace',
 'train_epochs': 100,
 'train_steps_per_epoch': 400,
 'train_learning_rate': 0.0004,
 'train_batch_size': 16,
 'train_tensorboard': True,
 'train_reduce_lr': {'factor': 0.5, 'patience': 10, 'min_delta': 0}}

In [10]:
model = CARE(config, 'my_model', basedir='models')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



/home/fadiqbal/anaconda3/lib/python3.7/site-packages/csbdeep/models/base_model.py:134: UserWarning: output path for model already exists, files may be overwritten: /home/fadiqbal/CSBDeep/examples/nucleoFadil/models/my_model
  warnings.warn('output path for model already exists, files may be overwritten: %s' % str(self.logdir.resolve()))


In [ ]:
history = model.train(X,Y, validation_data=(X_val,Y_val))


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




Epoch 1/100
400/400 [==============================] - 98s 246ms/step - loss: -0.6020 - mse: 0.0061 - mae: 0.0552 - val_loss: -0.9349 - val_mse: 0.0031 - val_mae: 0.0438

Epoch 2/100
400/400 [==============================] - 92s 230ms/step - loss: -1.5420 - mse: 0.0036 - mae: 0.0392 - val_loss: -0.8018 - val_mse: 0.0033 - val_mae: 0.0468
Epoch 3/100
400/400 [==============================] - 92s 231ms/step - loss: -1.8650 - mse: 0.0026 - mae: 0.0314 - val_loss: -1.9839 - val_mse: 0.0015 - val_mae: 0.0234
Epoch 4/100
400/400 [==============================] - 93s 232ms/step - loss: -2.2148 - mse: 0.0019 - mae: 0.0244 - val_l

In [ ]:
print(sorted(list(history.history.keys())))
plt.figure(figsize=(16,5))
plot_history(history,['loss','val_loss'],['mse','val_mse','mae','val_mae']);

In [ ]:
plt.figure(figsize=(12,10))
_P = model.keras_model.predict(X_val[:5])
_P_mean  = _P[...,:(_P.shape[-1]//2)]
_P_scale = _P[...,(_P.shape[-1]//2):]
plot_some(X_val[:5],Y_val[:5],_P_mean,_P_scale,pmax=99.5)
plt.suptitle('5 example validation patches\n'      
             'first row: input (source),  '        
             'second row: target (ground truth),  '
             'third row: predicted Laplace mean,  '
             'forth row: predicted Laplace scale');